<a href="https://colab.research.google.com/github/Kareem-1010/Climate-Disaster-Prediction-ST-GNN-/blob/main/Climate_gnn/notebooks/Climate(ST-GNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')   # allow saving files to your Drive
!nvidia-smi || true
import torch, sys
print("Python:", sys.version.splitlines()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)

Mounted at /content/drive
Wed Aug 13 06:45:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------

In [2]:

!pip install -q --upgrade pip
!pip install -q xarray netCDF4 rioxarray pandas numpy scipy scikit-learn cdsapi pydap earthengine-api matplotlib cartopy
# optional lighter geo libs if you need them later:
!pip install -q rasterio rioxarray geopandas shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.8 MB/s eta 0:00:00


In [3]:
import torch, sys
torch_version = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda
print("Detected torch:", torch_version, "CUDA:", cuda_ver)

Detected torch: 2.6.0 CUDA: 12.4


In [4]:
import sys, subprocess, importlib, traceback

def run(cmd):
    print("RUN:", " ".join(cmd))
    return subprocess.check_call(cmd)

try:
    import torch
    torch_version = torch.__version__.split('+')[0]
    cuda = torch.version.cuda
    if cuda is None:
        cuda_tag = "cpu"
    else:
        cuda_tag = "cu" + cuda.replace('.', '')
    index_url = f"https://data.pyg.org/whl/torch-{torch_version}+{cuda_tag}.html"
    print("Using PyG index:", index_url)
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch-scatter", "-f", index_url])
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch-sparse", "-f", index_url])
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch-cluster", "-f", index_url])
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch-spline-conv", "-f", index_url])
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch-geometric", "-f", index_url])

    import torch_geometric
    print("torch_geometric version:", torch_geometric.__version__)
except Exception as e:
    print("Automatic PyG install failed. Error:")
    traceback.print_exc()
    print("\nFALLBACK: If PyG install fails, you can either:\n"
          "- Try again after restarting the runtime (sometimes helps)\n"
          "- Use DGL instead for GNN experiments: `pip install dgl` (simpler install)\n"
          "- Or run the notebook on a local machine or Colab Pro where matching wheels exist.")

Using PyG index: https://data.pyg.org/whl/torch-2.6.0+cu124.html
RUN: /usr/bin/python3 -m pip install --quiet torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
RUN: /usr/bin/python3 -m pip install --quiet torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
RUN: /usr/bin/python3 -m pip install --quiet torch-cluster -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
RUN: /usr/bin/python3 -m pip install --quiet torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
RUN: /usr/bin/python3 -m pip install --quiet torch-geometric -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
torch_geometric version: 2.6.1


In [6]:
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize(project='climate-disaster-prediction')
print("Earth Engine initialized:", ee.data._initialized)

Earth Engine initialized: True


In [9]:
cds_key = input("Paste your CDS key in format <uid>:<key>  (press Enter when done): ").strip()
cds_path = "/root/.cdsapirc"
with open(cds_path, "w") as f:
    f.write("url: https://cds.climate.copernicus.eu/api/v2\n")
    f.write(f"key: {cds_key}\n")
print("Wrote CDS API config to", cds_path)

Paste your CDS key in format <uid>:<key>  (press Enter when done): 7b91d193-cd72-43c5-91bc-9ddd989f61b0:4a94b0bd-8861-4c55-ade8-11398dfac642
Wrote CDS API config to /root/.cdsapirc
